In [ ]:
%pip install pdfplumber pandas tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [8]:
import pdfplumber, re, pandas as pd

In [9]:

pdf_path = "National Grid Tariff-New York.pdf"

# --- Regex patterns ---
plan_pattern = re.compile(
    r"(Service\s+Class(?:ification)?\s+No\.?\s*\d+[A-Z\-]*\s*[–-]?\s*[A-Za-z0-9\s,&]*)",
    re.IGNORECASE
)
effective_pattern = re.compile(
    r"(?:Effective\s+(?:Date|on|from)|Issued)\s*[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})",
    re.IGNORECASE
)
rate_pattern = re.compile(
    r"(\$?\s?\d+\.\d{2}\s*(?:per|/)\s*(?:kWh|kW|day|month|therm)|\$\s?\d+\.\d{2})",
    re.IGNORECASE
)

results = []

with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if not text:
            continue

        lines = [l.strip() for l in text.split("\n") if l.strip()]

        # Get the last mentioned effective/issued date on this page
        eff_dates = effective_pattern.findall(text)
        current_effective = eff_dates[-1] if eff_dates else None

        current_plan = None

        for line in lines:
            # Detect new service plan name
            plan_match = plan_pattern.search(line)
            if plan_match:
                current_plan = plan_match.group(1).strip()

            # Detect tariff/rate lines
            rate_match = rate_pattern.findall(line)
            if rate_match and current_plan:
                results.append({
                    "Page": page_num,
                    "Plan Name": current_plan,
                    "Effective Date": current_effective,
                    "Tariff Line": line
                })

# Convert to DataFrame and clean duplicates
df = pd.DataFrame(results).drop_duplicates().reset_index(drop=True)

# --- Pretty printing ---
if df.empty:
    print("⚠️ No tariff data found — check if PDF is scanned or uses tables only.")
else:
    for plan in df["Plan Name"].unique():
        subset = df[df["Plan Name"] == plan]
        eff = subset["Effective Date"].iloc[0] or "Not Found"
        print(f"\n📘 {plan}")
        print(f"📅 Effective Date: {eff}")
        for _, row in subset.iterrows():
            print(f"💰 Tariff Line: {row['Tariff Line']}")


📘 Service Classification No. 1 or Service Classification No
📅 Effective Date: APRIL 27, 2009
💰 Tariff Line: establishing service or discontinuing service shall be assessed a charge of $20.00. The $20.00
💰 Tariff Line: 26.13.2 The $20.00 charge shall be applicable and limited to one charge per visit for the same
💰 Tariff Line: 26.13.3 A single $20.00 charge shall apply to multiple accounts for the same customer at the same
💰 Tariff Line: visit is exempt from the $20.00 charge.

📘 SERVICE CLASSIFICATION NO. 2-DEMAND
📅 Effective Date: OCTOBER 1, 2025
💰 Tariff Line: Distribution (per kW): $0.00 $4.25 $8.50 $12.74
💰 Tariff Line: On Peak (per kWh): $0.07420 $0.05565 $0.03710 $0.01855
💰 Tariff Line: Off Peak (per kWh): $0.03710 $0.02783 $0.01855 $0.00928
💰 Tariff Line: Super Peak (per kWh): $0.11131 $0.08348 $0.05565 $0.02783

📘 SERVICE CLASSIFICATION NO. 3
📅 Effective Date: OCTOBER 1, 2025
💰 Tariff Line: Distribution (per kW): $0.00 $3.57 $7.14 $10.71
💰 Tariff Line: On Peak (per kWh): $0.04

In [2]:
import pdfplumber
import re
import pandas as pd

pdf_path = "National Grid Tariff-New York.pdf"

# Regex patterns
plan_pattern = re.compile(r"(Service\s+Class(?:ification)?\s+No\.?\s*\d+[A-Z\-]*.*)", re.IGNORECASE)
effective_pattern = re.compile(r"(?:Effective|Issued)\s*[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})", re.IGNORECASE)

customer_charge_pattern = re.compile(r"(Customer\s+Charge[s]?:.*\$\s?[\d,]+(?:\.\d+)?)", re.IGNORECASE)
demand_charge_pattern = re.compile(r"((?:Demand|Distribution).*per\s*kW.*\$?\s?[\d,]+(?:\.\d+)?)", re.IGNORECASE)
energy_charge_pattern = re.compile(r"((?:On\s*Peak|Off\s*Peak|Super\s*Peak).*?\$\s?[\d,\.]+)", re.IGNORECASE)

# Storage
cust_charge_rows, demand_charge_rows, energy_charge_rows = [], [], []

with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if not text:
            continue

        plan_match = plan_pattern.search(text)
        plan = plan_match.group(1).strip() if plan_match else None

        dates = effective_pattern.findall(text)
        effective_date = dates[-1].strip() if dates else None

        for line in text.split("\n"):
            if customer_charge_pattern.search(line):
                cust_charge_rows.append({
                    "Page": page_num,
                    "Service Plan": plan,
                    "Effective Date": effective_date,
                    "Tariff Detail": line.strip()
                })
            elif demand_charge_pattern.search(line):
                demand_charge_rows.append({
                    "Page": page_num,
                    "Service Plan": plan,
                    "Effective Date": effective_date,
                    "Tariff Detail": line.strip()
                })
            elif energy_charge_pattern.search(line):
                energy_charge_rows.append({
                    "Page": page_num,
                    "Service Plan": plan,
                    "Effective Date": effective_date,
                    "Tariff Detail": line.strip()
                })

# Create DataFrames
df_cust = pd.DataFrame(cust_charge_rows).drop_duplicates().reset_index(drop=True)
df_demand = pd.DataFrame(demand_charge_rows).drop_duplicates().reset_index(drop=True)
df_energy = pd.DataFrame(energy_charge_rows).drop_duplicates().reset_index(drop=True)

# --- Display all tables directly in notebook ---
from IPython.display import display, HTML

print("🔹 CUSTOMER CHARGES TABLE")
display(HTML(df_cust.head(20).to_html(index=False)))

print("\n🔹 DEMAND CHARGES TABLE")
display(HTML(df_demand.head(20).to_html(index=False)))

print("\n🔹 ENERGY CHARGES TABLE")
display(HTML(df_energy.head(20).to_html(index=False)))

print(f"\n✅ Extraction complete! ({len(df_cust)} customer, {len(df_demand)} demand, {len(df_energy)} energy rows found.)")

# Uncomment these lines when ready to save
# df_cust.to_csv("cust_charges.csv", index=False)
# df_demand.to_csv("demand_charges.csv", index=False)
# df_energy.to_csv("energy_charges.csv", index=False)


🔹 CUSTOMER CHARGES TABLE


Page,Service Plan,Effective Date,Tariff Detail
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Customer Charge: $17.33
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Customer Charge: $19.00
625,SERVICE CLASSIFICATION NO. 7 (Continued),None,Customer Charge: $21.02
625,SERVICE CLASSIFICATION NO. 7 (Continued),None,Customer Charge: $25.00
626,SERVICE CLASSIFICATION NO. 7 (Continued),None,Customer Charge: $65.00



🔹 DEMAND CHARGES TABLE


Page,Service Plan,Effective Date,Tariff Detail
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Distribution (per kW): $0.00 $4.25 $8.50 $12.74
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Distribution (per kW): $0.00 $3.57 $7.14 $10.71
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Distribution (per kW): $0.00 $3.22 $6.44 $9.66
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Distribution (per kW): $0.00 $1.02 $2.04 $3.05
341,SERVICE CLASSIFICATION NO. 3A (delivery voltage 0-2.2 kV),None,Distribution (per kW): $0.00 $3.52 $7.04 $10.55
341,SERVICE CLASSIFICATION NO. 3A (delivery voltage 0-2.2 kV),None,Distribution (per kW): $0.00 $1.24 $2.49 $3.73
341,SERVICE CLASSIFICATION NO. 3A (delivery voltage 0-2.2 kV),None,Distribution (per kW): $0.00 $1.09 $2.18 $3.27
349,None,None,"customers, and on a Contract Demand per kW basis for SC7 customers, where applicable."
424,None,None,"for non-demand service classes, on a per kW basis for demand service classes, and on a Contract"
568,SERVICE CLASSIFICATION NO. 3 (Continued),"June 1, 2013",Additional Demand Charges: Per kW of Demand for all kW of Demand over 40:



🔹 ENERGY CHARGES TABLE


Page,Service Plan,Effective Date,Tariff Detail
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,On Peak (per kWh): $0.07420 $0.05565 $0.03710 $0.01855
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Off Peak (per kWh): $0.03710 $0.02783 $0.01855 $0.00928
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Super Peak (per kWh): $0.11131 $0.08348 $0.05565 $0.02783
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,On Peak (per kWh): $0.04805 $0.03604 $0.02403 $0.01201
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Off Peak (per kWh): $0.02403 $0.01802 $0.01201 $0.00601
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Super Peak (per kWh): $0.07208 $0.05406 $0.03604 $0.01802
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,On Peak (per kWh): $0.03984 $0.02988 $0.01992 $0.00996
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Off Peak (per kWh): $0.01992 $0.01494 $0.00996 $0.00498
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Super Peak (per kWh): $0.05976 $0.04482 $0.02988 $0.01494
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,On Peak (per kWh): $0.01257 $0.00943 $0.00629 $0.00314



✅ Extraction complete! (5 customer, 11 demand, 23 energy rows found.)


In [4]:
import pdfplumber, re, pandas as pd
from tabulate import tabulate
from IPython.display import display, HTML

# STEP 1 ── Path to your tariff PDF
pdf_path = "National Grid Tariff-New York.pdf"

# STEP 2 ── Regex patterns
plan_pattern       = re.compile(r"(Service\s+Class(?:ification)?\s+No\.?\s*\d+[A-Z\-]*.*)", re.IGNORECASE)
effective_pattern  = re.compile(r"(?:Effective|Issued)\s*[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})", re.IGNORECASE)
rate_line_pattern  = re.compile(r"(\$?\s?\d+\.\d{2}.*?(?:kWh|kW|month|RkVA|customer|charge))", re.IGNORECASE)

# For charge-type tagging
def classify_condition(line):
    if re.search(r"per\s*kWh", line, re.IGNORECASE):   return "Energy Charge"
    if re.search(r"per\s*kW", line, re.IGNORECASE):    return "Demand Charge"
    if re.search(r"per\s*month|Customer\s*Charge", line, re.IGNORECASE): return "Fixed Charge"
    if re.search(r"Reactive|RkVA", line, re.IGNORECASE): return "Power Factor Charge"
    return "Other"

records = []

# STEP 3 ── Extract from every page
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if not text:
            continue
        
        plan_match = plan_pattern.search(text)
        plan = plan_match.group(1).strip() if plan_match else None
        
        eff_dates = effective_pattern.findall(text)
        effective_date = eff_dates[-1].strip() if eff_dates else None
        
        for line in text.split("\n"):
            if rate_line_pattern.search(line):
                records.append({
                    "Page": page_num,
                    "Service Plan": plan,
                    "Effective Date": effective_date,
                    "Tariff Line": line.strip(),
                    "Charge Type": classify_condition(line)
                })

# STEP 4 ── Create DataFrame
df = pd.DataFrame(records).drop_duplicates().reset_index(drop=True)

# STEP 5 ── Split by charge type
df_energy = df[df["Charge Type"] == "Energy Charge"].reset_index(drop=True)
df_demand = df[df["Charge Type"] == "Demand Charge"].reset_index(drop=True)
df_fixed  = df[df["Charge Type"] == "Fixed Charge"].reset_index(drop=True)
df_other  = df[df["Charge Type"].isin(["Power Factor Charge", "Other"])].reset_index(drop=True)

# STEP 6 ── Display clean previews in the notebook
print("🔹 ENERGY CHARGES (first 15 rows)")
display(HTML(df_energy.head(15).to_html(index=False)))

print("🔹 DEMAND CHARGES (first 15 rows)")
display(HTML(df_demand.head(15).to_html(index=False)))

print("🔹 FIXED (CUSTOMER) CHARGES (first 15 rows)")
display(HTML(df_fixed.head(15).to_html(index=False)))

print("🔹 OTHER / POWER FACTOR CHARGES (first 10 rows)")
display(HTML(df_other.head(10).to_html(index=False)))

print(f"\n✅ Extraction complete!  {len(df)} total rate lines found "
      f"({len(df_energy)} energy, {len(df_demand)} demand, "
      f"{len(df_fixed)} fixed, {len(df_other)} other).")

# STEP 7 ── (Optional) Save for later merging with bills
# df_energy.to_csv("energy_charges.csv", index=False)
# df_demand.to_csv("demand_charges.csv", index=False)
# df_fixed.to_csv("fixed_charges.csv", index=False)
# df_other.to_csv("other_charges.csv", index=False)


🔹 ENERGY CHARGES (first 15 rows)


Page,Service Plan,Effective Date,Tariff Line,Charge Type
408,None,"April 23, 2018",62.10.1.2.1 Dollars per kWh for Load Relief provided during a Planned Event;,Energy Charge
408,None,"April 23, 2018",62.10.1.2.2 Dollars per kWh for Load Relief provided during an Unplanned,Energy Charge


🔹 DEMAND CHARGES (first 15 rows)


Page,Service Plan,Effective Date,Tariff Line,Charge Type


🔹 FIXED (CUSTOMER) CHARGES (first 15 rows)


Page,Service Plan,Effective Date,Tariff Line,Charge Type
388,None,None,"the amount of $14.56 per month to cover the incremental cost of metering communications, until",Fixed Charge
404,None,None,Customer Charge in the amount of $14.56 per month to cover the incremental cost of metering,Fixed Charge
417,None,None,Customer Charge in the amount of $14.56 per month to cover the incremental cost of metering,Fixed Charge
507,None,None,pay the Company an Incremental Customer Charge in the amount of $14.56 per month to cover the incremental cost,Fixed Charge
514,None,None,Incremental Customer Charge in the amount of $14.56 per month to cover the incremental cost of metering,Fixed Charge
524,None,None,Incremental Customer Charge in the amount of $14.56 per month to cover the incremental cost of metering,Fixed Charge
537,SERVICE CLASSIFICATION NO. 1 (Continued),None,incremental customer charge of $4.42 per month. The incremental customer charge will be waived for,Fixed Charge
566,SERVICE CLASSIFICATION NO. 2 (Continued),"June 1, 2013",incremental customer charge of $25.10/month.,Fixed Charge
578,SERVICE CLASSIFICATION NO. 3 (Continued),None,"incremental customer charge of $25.10/month, except Service Classification No. 12 customers who have",Fixed Charge
579,SERVICE CLASSIFICATION NO. 3 (Continued),"June 1, 2013",incremental customer charge of $25.10/month.,Fixed Charge


🔹 OTHER / POWER FACTOR CHARGES (first 10 rows)


Page,Service Plan,Effective Date,Tariff Line,Charge Type
26,None,None,"1.5.10 A ""Delinquent Customer"" is a customer who has made a late payment on two or more",Other
26,None,None,"1.5.13 A ""Levelized Payment Plan"" is a billing plan designed to reduce fluctuation in a customer's",Other
26,None,None,"1.5.14 A ""Backbill"" is that portion of any bill, other than a levelized bill, which represents charges",Other
27,None,None,"1.11 ""Cost"" or ""expense"" shall include all labor, material and other charges applicable thereto, including cost",Other
28,None,None,"1.12 ""Surcharge"" is a charge payable by Customer to Company, in addition to the charge for electric service",Other
28,None,None,"1.15 ""Standby Service"" is service which is supplied to a Customer who obtains all or a part of the customer’s",Other
30,None,None,"1.20 ""On-Site Generation"" - Any facility which is capable of generating electricity for use by a Customer",Other
30,None,None,"1.26 ""Energy Service Company"" (ESCo) - Any non-utility entity that can perform energy and customer",Other
30,None,None,"1.27 ""Electricity Supply Service"" (ESS) - The furnishing of the electricity required to meet a Customer's",Other
30,None,None,"1.28 ""Slamming"" - The unauthorized transfer of a Customer's account from one entity providing Electricity",Other



✅ Extraction complete!  148 total rate lines found (2 energy, 0 demand, 11 fixed, 135 other).


In [5]:
import pdfplumber, re, pandas as pd
from IPython.display import display, HTML

pdf_path = "National Grid Tariff-New York.pdf"

# ── Regex patterns
plan_pattern      = re.compile(r"(Service\s+Class(?:ification)?\s+No\.?\s*\d+[A-Z\-]*.*)", re.IGNORECASE)
effective_pattern = re.compile(r"(?:Effective|Issued)\s*[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})", re.IGNORECASE)
rate_line_pattern = re.compile(r"\$\s?\d+\.\d{2}", re.IGNORECASE)

# Helper: identify the type of rate and distribution condition
def detect_charge_type(line):
    if "per kwh" in line.lower(): return "Energy Charge"
    if "per kw" in line.lower(): return "Demand Charge"
    if "customer charge" in line.lower() or "per month" in line.lower(): return "Fixed Charge"
    if "reactive" in line.lower() or "rkva" in line.lower(): return "Power Factor Charge"
    return "Other"

def detect_distribution(line):
    if "secondary" in line.lower(): return "Secondary (<2.2 kV)"
    if "primary" in line.lower(): return "Primary (2.2–15 kV)"
    if "sub" in line.lower(): return "Sub-Transmission (22–50 kV)"
    if "transmission" in line.lower(): return "Transmission (>60 kV)"
    return "All Voltages / Unspecified"

records = []

# ── Read every page
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if not text:
            continue

        plan = None
        eff_date = None
        plan_match = plan_pattern.search(text)
        if plan_match:
            plan = plan_match.group(1).strip()

        dates = effective_pattern.findall(text)
        if dates:
            eff_date = dates[-1].strip()

        for line in text.split("\n"):
            if rate_line_pattern.search(line):
                rate_values = re.findall(r"\$?\d+\.\d{2}", line)
                # pad to always have 4 columns (Secondary, Primary, Sub-Transmission, Transmission)
                while len(rate_values) < 4:
                    rate_values.append(None)

                records.append({
                    "Page": page_num,
                    "Service Plan": plan,
                    "Effective Date": eff_date,
                    "Charge Type": detect_charge_type(line),
                    "Distribution Level": detect_distribution(line),
                    "Rate_Secondary": rate_values[0],
                    "Rate_Primary": rate_values[1],
                    "Rate_SubTrans": rate_values[2],
                    "Rate_Trans": rate_values[3],
                    "Raw Text": line.strip()
                })

df = pd.DataFrame(records).drop_duplicates().reset_index(drop=True)

# ── Separate by charge type
df_energy = df[df["Charge Type"] == "Energy Charge"]
df_demand = df[df["Charge Type"] == "Demand Charge"]
df_fixed  = df[df["Charge Type"] == "Fixed Charge"]
df_other  = df[df["Charge Type"].isin(["Power Factor Charge","Other"])]

# ── Display previews
print("🔹 ENERGY CHARGES (first 10)")
display(HTML(df_energy.head(10).to_html(index=False)))

print("🔹 DEMAND CHARGES (first 10)")
display(HTML(df_demand.head(10).to_html(index=False)))

print("🔹 FIXED CHARGES (first 10)")
display(HTML(df_fixed.head(10).to_html(index=False)))

print("🔹 OTHER CHARGES (first 10)")
display(HTML(df_other.head(10).to_html(index=False)))

print(f"\n✅ Extraction complete. {len(df)} total detailed rate lines found.")


🔹 ENERGY CHARGES (first 10)


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Energy Charge,All Voltages / Unspecified,$0.07,$0.05,$0.03,$0.01,On Peak (per kWh): $0.07420 $0.05565 $0.03710 $0.01855
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Energy Charge,All Voltages / Unspecified,$0.03,$0.02,$0.01,$0.00,Off Peak (per kWh): $0.03710 $0.02783 $0.01855 $0.00928
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Energy Charge,All Voltages / Unspecified,$0.11,$0.08,$0.05,$0.02,Super Peak (per kWh): $0.11131 $0.08348 $0.05565 $0.02783
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Energy Charge,All Voltages / Unspecified,$0.04,$0.03,$0.02,$0.01,On Peak (per kWh): $0.04805 $0.03604 $0.02403 $0.01201
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Energy Charge,All Voltages / Unspecified,$0.02,$0.01,$0.01,$0.00,Off Peak (per kWh): $0.02403 $0.01802 $0.01201 $0.00601
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Energy Charge,All Voltages / Unspecified,$0.07,$0.05,$0.03,$0.01,Super Peak (per kWh): $0.07208 $0.05406 $0.03604 $0.01802
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Energy Charge,All Voltages / Unspecified,$0.03,$0.02,$0.01,$0.00,On Peak (per kWh): $0.03984 $0.02988 $0.01992 $0.00996
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Energy Charge,All Voltages / Unspecified,$0.01,$0.01,$0.00,$0.00,Off Peak (per kWh): $0.01992 $0.01494 $0.00996 $0.00498
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Energy Charge,All Voltages / Unspecified,$0.05,$0.04,$0.02,$0.01,Super Peak (per kWh): $0.05976 $0.04482 $0.02988 $0.01494
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Energy Charge,All Voltages / Unspecified,$0.01,$0.00,$0.00,$0.00,On Peak (per kWh): $0.01257 $0.00943 $0.00629 $0.00314


🔹 DEMAND CHARGES (first 10)


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Demand Charge,All Voltages / Unspecified,$0.00,$4.25,$8.50,$12.74,Distribution (per kW): $0.00 $4.25 $8.50 $12.74
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Demand Charge,All Voltages / Unspecified,$0.00,$3.57,$7.14,$10.71,Distribution (per kW): $0.00 $3.57 $7.14 $10.71
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Demand Charge,All Voltages / Unspecified,$0.00,$3.22,$6.44,$9.66,Distribution (per kW): $0.00 $3.22 $6.44 $9.66
340,SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV),None,Demand Charge,All Voltages / Unspecified,$0.00,$1.02,$2.04,$3.05,Distribution (per kW): $0.00 $1.02 $2.04 $3.05
341,SERVICE CLASSIFICATION NO. 3A (delivery voltage 0-2.2 kV),None,Demand Charge,All Voltages / Unspecified,$0.00,$3.52,$7.04,$10.55,Distribution (per kW): $0.00 $3.52 $7.04 $10.55
341,SERVICE CLASSIFICATION NO. 3A (delivery voltage 0-2.2 kV),None,Demand Charge,All Voltages / Unspecified,$0.00,$1.24,$2.49,$3.73,Distribution (per kW): $0.00 $1.24 $2.49 $3.73
341,SERVICE CLASSIFICATION NO. 3A (delivery voltage 0-2.2 kV),None,Demand Charge,All Voltages / Unspecified,$0.00,$1.09,$2.18,$3.27,Distribution (per kW): $0.00 $1.09 $2.18 $3.27
557,SERVICE CLASSIFICATION NO. 2 (Continued),None,Demand Charge,All Voltages / Unspecified,$16.99,None,None,None,per kW: $16.99
581,SERVICE CLASSIFICATION NO. 3A (Continued),None,Demand Charge,All Voltages / Unspecified,$14.07,$14.07,$4.97,$4.36,Charges; per kW: $14.07 $14.07 $4.97 $4.36


🔹 FIXED CHARGES (first 10)


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
388,None,None,Fixed Charge,All Voltages / Unspecified,$14.56,None,None,None,"the amount of $14.56 per month to cover the incremental cost of metering communications, until"
404,None,None,Fixed Charge,All Voltages / Unspecified,$14.56,None,None,None,Customer Charge in the amount of $14.56 per month to cover the incremental cost of metering
417,None,None,Fixed Charge,All Voltages / Unspecified,$14.56,None,None,None,Customer Charge in the amount of $14.56 per month to cover the incremental cost of metering
507,None,None,Fixed Charge,All Voltages / Unspecified,$14.56,None,None,None,pay the Company an Incremental Customer Charge in the amount of $14.56 per month to cover the incremental cost
514,None,None,Fixed Charge,All Voltages / Unspecified,$14.56,None,None,None,Incremental Customer Charge in the amount of $14.56 per month to cover the incremental cost of metering
524,None,None,Fixed Charge,All Voltages / Unspecified,$14.56,None,None,None,Incremental Customer Charge in the amount of $14.56 per month to cover the incremental cost of metering
537,SERVICE CLASSIFICATION NO. 1 (Continued),None,Fixed Charge,All Voltages / Unspecified,$4.42,None,None,None,incremental customer charge of $4.42 per month. The incremental customer charge will be waived for
565,SERVICE CLASSIFICATION NO. 2 (CONTINUED),"June 1, 2012",Fixed Charge,All Voltages / Unspecified,$4.42,None,None,None,incremental customer charge of $4.42/mo.
566,SERVICE CLASSIFICATION NO. 2 (Continued),"June 1, 2013",Fixed Charge,All Voltages / Unspecified,$25.10,None,None,None,incremental customer charge of $25.10/month.
578,SERVICE CLASSIFICATION NO. 3 (Continued),None,Fixed Charge,All Voltages / Unspecified,$25.10,None,None,None,"incremental customer charge of $25.10/month, except Service Classification No. 12 customers who have"


🔹 OTHER CHARGES (first 10)


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
46,None,None,Other,All Voltages / Unspecified,$25.00,None,None,None,shall pay to the applicant the sum of twenty-five dollars ($25.00) per day for each day that service is not
84,None,None,Other,All Voltages / Unspecified,$80.00,None,None,None,9.1.1.1.1 $80.00 when the service was disconnected at the meter.
84,None,None,Other,All Voltages / Unspecified,$230.00,None,None,None,9.1.1.1.2 $230.00 when the service was disconnected at the pole.
84,None,None,Other,All Voltages / Unspecified,$96.00,None,None,None,9.1.1.2.1 $96.00 when the service was disconnected at the meter.
84,None,None,Other,All Voltages / Unspecified,$405.00,None,None,None,9.1.1.2.2 $405.00 when the service was disconnected at the pole.
85,None,None,Other,All Voltages / Unspecified,$80.00,None,None,None,9.2.1.1.1 $80.00 when the service was re-established or disconnected at the
85,None,None,Other,All Voltages / Unspecified,$230.00,None,None,None,9.2.1.1.2 $230.00 when the service was re-established or disconnected at the
85,None,None,Other,All Voltages / Unspecified,$96.00,None,None,None,9.2.1.2.1 $96.00 when the service was re-established or disconnected at the
85,None,None,Other,All Voltages / Unspecified,$405.00,None,None,None,9.2.1.2.2 $405.00 when the service was re-established or disconnected at the
85,None,None,Other,All Voltages / Unspecified,$23.00,None,None,None,9.2.2.1.1 $23.00 when the service was re-established or disconnected at the



✅ Extraction complete. 182 total detailed rate lines found.


In [8]:
from IPython.display import display, HTML

# 🧾 List all unique Service Classifications
unique_plans = df["Service Plan"].dropna().unique()
print(f"📘 Found {len(unique_plans)} Service Classifications:\n")
for i, plan in enumerate(unique_plans, start=1):
    print(f"{i}. {plan}")

# 🔍 Display data for each Service Classification
for plan in unique_plans:
    subset = df[df["Service Plan"] == plan].reset_index(drop=True)
    print("\n" + "="*120)
    print(f"📘 {plan}")
    if subset['Effective Date'].notna().any():
        eff_dates = subset['Effective Date'].dropna().unique()
        print(f"📅 Effective Dates Found: {', '.join(eff_dates)}")
    else:
        print("📅 Effective Dates Found: None")
    
    print(f"💰 Total Tariff Lines: {len(subset)}")
    display(HTML(subset.head(20).to_html(index=False)))  # Show first 20 rows nicely


📘 Found 16 Service Classifications:

1. Service Classification No. 1 or Service Classification No. 2
2. Service Classification No. 3 and 3A customers, the Distribution Delivery
3. SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV)
4. SERVICE CLASSIFICATION NO. 3A (delivery voltage 0-2.2 kV)
5. SERVICE CLASSIFICATION NO. 1
6. SERVICE CLASSIFICATION NO. 1 (Continued)
7. SERVICE CLASSIFICATION NO. 1-C (Continued)
8. SERVICE CLASSIFICATION NO. 2
9. SERVICE CLASSIFICATION NO. 2 (Continued)
10. SERVICE CLASSIFICATION NO. 2 (CONTINUED)
11. SERVICE CLASSIFICATION NO. 3 (Continued)
12. SERVICE CLASSIFICATION NO. 3A (Continued)
13. SERVICE CLASSIFICATION NO. 4 (Continued)
14. SERVICE CLASSIFICATION NO. 6 (Continued)
15. SERVICE CLASSIFICATION NO. 7 (Continued)
16. SERVICE CLASSIFICATION NO. 7 (CONTINUED)

📘 Service Classification No. 1 or Service Classification No. 2
📅 Effective Dates Found: None
💰 Total Tariff Lines: 4


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
155,Service Classification No. 1 or Service Classification No. 2,None,Other,All Voltages / Unspecified,$20.00,$20.00,None,None,establishing service or discontinuing service shall be assessed a charge of $20.00. The $20.00
155,Service Classification No. 1 or Service Classification No. 2,None,Other,All Voltages / Unspecified,26.13,$20.00,None,None,26.13.2 The $20.00 charge shall be applicable and limited to one charge per visit for the same
155,Service Classification No. 1 or Service Classification No. 2,None,Other,All Voltages / Unspecified,26.13,$20.00,None,None,26.13.3 A single $20.00 charge shall apply to multiple accounts for the same customer at the same
155,Service Classification No. 1 or Service Classification No. 2,None,Other,All Voltages / Unspecified,$20.00,None,None,None,visit is exempt from the $20.00 charge.



📘 Service Classification No. 3 and 3A customers, the Distribution Delivery
📅 Effective Dates Found: None
💰 Total Tariff Lines: 4


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Demand Charge,All Voltages / Unspecified,$0.00,$4.25,$8.50,$12.74,Distribution (per kW): $0.00 $4.25 $8.50 $12.74
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Energy Charge,All Voltages / Unspecified,$0.07,$0.05,$0.03,$0.01,On Peak (per kWh): $0.07420 $0.05565 $0.03710 $0.01855
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Energy Charge,All Voltages / Unspecified,$0.03,$0.02,$0.01,$0.00,Off Peak (per kWh): $0.03710 $0.02783 $0.01855 $0.00928
339,"Service Classification No. 3 and 3A customers, the Distribution Delivery",None,Energy Charge,All Voltages / Unspecified,$0.11,$0.08,$0.05,$0.02,Super Peak (per kWh): $0.11131 $0.08348 $0.05565 $0.02783



📘 SERVICE CLASSIFICATION NO. 3 (delivery voltage 0-2.2 kV)
📅 Effective Dates Found: None
💰 Total Tariff Lines: 12


...

Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,All Voltages / Unspecified,$19.00,$5.49,None,None,SC1 $19.00 $5.49
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,All Voltages / Unspecified,$25.00,$5.91,None,None,SC2ND $25.00 $5.91
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,All Voltages / Unspecified,$65.00,$2.91,None,None,SC2D $65.00 $2.91
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,Secondary (<2.2 kV),$775.00,$0.21,None,None,SC3 Secondary $775.00 $0.21
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,Primary (2.2–15 kV),$850.00,$0.00,None,None,SC3 Primary $850.00 $0.00
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,Sub-Transmission (22–50 kV),430.00,$0.00,None,None,"SC3 Subtransmission $1,430.00 $0.00"
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,Transmission (>60 kV),430.00,$0.00,None,None,"SC3 Transmission $1,430.00 $0.00"
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,Secondary (<2.2 kV),525.00,$0.00,None,None,"SC3A Secondary $3,525.00 $0.00"
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,Primary (2.2–15 kV),525.00,$0.00,None,None,"SC3A Primary $3,525.00 $0.00"
610,SERVICE CLASSIFICATION NO. 6 (Continued),None,Other,Sub-Transmission (22–50 kV),350.00,$0.00,None,None,"SC3A Subtransmission $4,350.00 $0.00"



📘 SERVICE CLASSIFICATION NO. 7 (Continued)
📅 Effective Dates Found: None
💰 Total Tariff Lines: 40


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Fixed Charge,All Voltages / Unspecified,$17.33,None,None,None,Customer Charge: $17.33
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Other,All Voltages / Unspecified,$28.81,None,None,None,Distribution Delivery $28.81
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Other,All Voltages / Unspecified,$0.24,None,None,None,Distribution Delivery $0.2448
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Fixed Charge,All Voltages / Unspecified,$19.00,None,None,None,Customer Charge: $19.00
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Other,All Voltages / Unspecified,$5.49,None,None,None,Distribution Delivery $5.49
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Other,All Voltages / Unspecified,$0.46,None,None,None,Distribution Delivery $0.4610
624,SERVICE CLASSIFICATION NO. 7 (Continued),None,Other,All Voltages / Unspecified,$1.01,None,None,None,Distribution Delivery $1.0179
625,SERVICE CLASSIFICATION NO. 7 (Continued),None,Fixed Charge,All Voltages / Unspecified,$21.02,None,None,None,Customer Charge: $21.02
625,SERVICE CLASSIFICATION NO. 7 (Continued),None,Other,All Voltages / Unspecified,$22.36,None,None,None,Distribution Delivery $22.36
625,SERVICE CLASSIFICATION NO. 7 (Continued),None,Other,All Voltages / Unspecified,$0.27,None,None,None,Distribution Delivery $0.2751



📘 SERVICE CLASSIFICATION NO. 7 (CONTINUED)
📅 Effective Dates Found: None
💰 Total Tariff Lines: 1


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_Secondary,Rate_Primary,Rate_SubTrans,Rate_Trans,Raw Text
646,SERVICE CLASSIFICATION NO. 7 (CONTINUED),None,Other,All Voltages / Unspecified,$0.40,$0.40,$0.00,$0.00,Distribution Delivery $0.4017 $0.4017 $0.0000 $0.0000


In [10]:
import pdfplumber, re, pandas as pd, numpy as np
from IPython.display import display, HTML

pdf_path = "National Grid Tariff-New York.pdf"

# Regex patterns
plan_pattern = re.compile(r"(Service\s+Class(?:ification)?\s+No\.?\s*\d+[A-Z\-]*.*)", re.IGNORECASE)
effective_pattern = re.compile(r"(?:Effective|Issued)\s*[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})", re.IGNORECASE)
money_pattern = re.compile(r"\$?\s?\d{1,3}(?:,\d{3})*(?:\.\d{1,4})?", re.IGNORECASE)

# Helper functions
def detect_charge_type(line):
    text = line.lower()
    if "per kwh" in text: return "Energy Charge"
    if "per kw" in text: return "Demand Charge"
    if "customer charge" in text or "per month" in text: return "Fixed Charge"
    if "rkva" in text or "reactive" in text: return "Power Factor Charge"
    return "Other"

def detect_distribution(line):
    t = line.lower()
    if "secondary" in t: return "Secondary (<2.2 kV)"
    if "primary" in t: return "Primary (2.2–15 kV)"
    if "sub" in t: return "Sub-Transmission (22–50 kV)"
    if "transmission" in t: return "Transmission (>60 kV)"
    return "All Voltages / Unspecified"

records = []

# ------------------- MAIN EXTRACTION -------------------
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text() or ""
        plan_match = plan_pattern.search(text)
        plan = plan_match.group(1).strip() if plan_match else None

        eff_match = effective_pattern.findall(text)
        eff_date = eff_match[-1].strip() if eff_match else None

        # --- TEXT-BASED LINES ---
        for line in text.split("\n"):
            if "$" in line or "per" in line.lower():
                rate_values = money_pattern.findall(line)
                if rate_values:
                    while len(rate_values) < 4:
                        rate_values.append(None)
                    records.append({
                        "Page": page_num,
                        "Service Plan": plan,
                        "Effective Date": eff_date,
                        "Charge Type": detect_charge_type(line),
                        "Distribution Level": detect_distribution(line),
                        "Rate_1": rate_values[0],
                        "Rate_2": rate_values[1],
                        "Rate_3": rate_values[2],
                        "Rate_4": rate_values[3],
                        "Source": "Text",
                        "Raw Text": line.strip()
                    })

        # --- TABLE-BASED EXTRACTION ---
        for table in page.extract_tables() or []:
            df_table = pd.DataFrame(table)
            df_table = df_table.replace(r'^\s*$', np.nan, regex=True).dropna(how="all")
            for _, row in df_table.iterrows():
                row_text = " | ".join([str(x) for x in row if pd.notna(x)])
                if "$" in row_text or "per" in row_text.lower():
                    rate_values = money_pattern.findall(row_text)
                    while len(rate_values) < 4:
                        rate_values.append(None)
                    records.append({
                        "Page": page_num,
                        "Service Plan": plan,
                        "Effective Date": eff_date,
                        "Charge Type": detect_charge_type(row_text),
                        "Distribution Level": detect_distribution(row_text),
                        "Rate_1": rate_values[0],
                        "Rate_2": rate_values[1],
                        "Rate_3": rate_values[2],
                        "Rate_4": rate_values[3],
                        "Source": "Table",
                        "Raw Text": row_text
                    })

# ------------------- CLEANUP -------------------
df_full = pd.DataFrame(records).drop_duplicates().reset_index(drop=True)

# Fill forward missing Service Plan / Effective Date (when same page continues)
df_full["Service Plan"] = df_full["Service Plan"].ffill()
df_full["Effective Date"] = df_full["Effective Date"].ffill()

# Drop rows with no money values
df_full = df_full[df_full[["Rate_1","Rate_2","Rate_3","Rate_4"]].notna().any(axis=1)]

# Display output summary
print(f"✅ Extracted {len(df_full)} tariff rate lines "
      f"from {df_full['Service Plan'].nunique()} service classifications.")

display(HTML(df_full.head(20).to_html(index=False)))


✅ Extracted 1565 tariff rate lines from 57 service classifications.


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_1,Rate_2,Rate_3,Rate_4,Source,Raw Text
1,None,"April 27, 2009",Other,All Voltages / Unspecified,27,200,9,None,Text,"INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"
1,None,"April 27, 2009",Other,All Voltages / Unspecified,207,None,None,None,Text,SUPERSEDING P.S.C. No. 207
1,None,"April 27, 2009",Other,All Voltages / Unspecified,220,220,None,None,Text,"P.S.C. No. 220 Schedule for Electric Service (""P.S.C. No. 220 Electricity Tariff"") supersedes and replaces former"
2,None,"April 27, 2009",Other,All Voltages / Unspecified,1,202,5,17,Text,"INITIAL EFFECTIVE DATE: SEPTEMBER 1, 2025 SUPERSEDING REVISION: 17"
2,None,"April 27, 2009",Other,All Voltages / Unspecified,28,146,None,None,Text,28. Special Services Performed by Company for Customer at a Charge 146
3,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,1,202,5,31,Text,"INITIAL EFFECTIVE DATE: OCTOBER 1, 2025 SUPERSEDING REVISION: 31"
4,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,1,202,4,4,Text,"INITIAL EFFECTIVE DATE: JANUARY 1, 2024 SUPERSEDING REVISION: 4"
5,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,27,200,9,None,Text,"INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"
6,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,27,200,9,None,Text,"INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"
7,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,27,200,9,None,Text,"INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"


In [11]:
display(HTML(df_full.to_html(index=False)))


Page,Service Plan,Effective Date,Charge Type,Distribution Level,Rate_1,Rate_2,Rate_3,Rate_4,Source,Raw Text
1,None,"April 27, 2009",Other,All Voltages / Unspecified,27,200,9,None,Text,"INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"
1,None,"April 27, 2009",Other,All Voltages / Unspecified,207,None,None,None,Text,SUPERSEDING P.S.C. No. 207
1,None,"April 27, 2009",Other,All Voltages / Unspecified,220,220,None,None,Text,"P.S.C. No. 220 Schedule for Electric Service (""P.S.C. No. 220 Electricity Tariff"") supersedes and replaces former"
2,None,"April 27, 2009",Other,All Voltages / Unspecified,1,202,5,17,Text,"INITIAL EFFECTIVE DATE: SEPTEMBER 1, 2025 SUPERSEDING REVISION: 17"
2,None,"April 27, 2009",Other,All Voltages / Unspecified,28,146,None,None,Text,28. Special Services Performed by Company for Customer at a Charge 146
3,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,1,202,5,31,Text,"INITIAL EFFECTIVE DATE: OCTOBER 1, 2025 SUPERSEDING REVISION: 31"
4,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,1,202,4,4,Text,"INITIAL EFFECTIVE DATE: JANUARY 1, 2024 SUPERSEDING REVISION: 4"
5,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,27,200,9,None,Text,"INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"
6,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,27,200,9,None,Text,"INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"
7,"Service Classification No. 12 - Form ""I"" 290","April 27, 2009",Other,All Voltages / Unspecified,27,200,9,None,Text,"INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"


In [1]:
%pip install pdfplumber camelot-py[cv] pandas pytesseract pdf2image numpy


  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.9 MB ? eta -:--:--
    --------------------------------------- 0.5/38.9 MB 837.5 kB/s eta 0:00:46
    --------------------------------------- 0.5/38.9 MB 837.5 kB/s eta 0:00:46
    --------------------------------------- 0.8/38.9 MB 958.5 kB/s eta 0:00:40
   - -------------------------------------- 1.0/38.9 MB 882.6 kB/s eta 0:00:43
   - -------------------------------------- 1.0/38.9 MB 882.6 kB/s eta 0:00:43
   - -------------------------------------- 1.3/38.9 MB 871.6 kB/s eta 0:00:44
   - -------------------------------------- 1.6/38.9 MB 921.7 kB/s eta 0:00:41
   - -------------------------------------- 1.8/38.9 MB 967.9 kB/s eta 0:00:39
   -- --------------

In [2]:
# Import all libraries
import os, re, pdfplumber, pandas as pd, numpy as np
from pdf2image import convert_from_path
import pytesseract
import camelot
from IPython.display import display, HTML

# -------------------------------
# CONFIGURATION
# -------------------------------
PDF_PATH = "National Grid Tariff-New York.pdf"  # Upload your tariff PDF first
records = []

# -------------------------------
# REGEX PATTERNS
# -------------------------------
PLAN_PATTERN = re.compile(r"(Service\s+Class(?:ification)?\s+No\.?\s*\d+[A-Z\-]*.*)", re.IGNORECASE)
EFFECTIVE_PATTERN = re.compile(r"(?:Effective|Issued)\s*[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})", re.IGNORECASE)
MONEY_PATTERN = re.compile(r"\$?\d+(?:\.\d{1,4})?")

def classify_charge_type(line):
    t = line.lower()
    if "per kwh" in t: return "Energy Charge"
    if "per kw" in t: return "Demand Charge"
    if "per month" in t or "customer charge" in t: return "Fixed Charge"
    if "reactive" in t or "rkva" in t: return "Power Factor Charge"
    return "Other"

def classify_voltage(line):
    t = line.lower()
    if "secondary" in t: return "Secondary (<2.2 kV)"
    if "primary" in t: return "Primary (2.2–15 kV)"
    if "sub" in t: return "Sub-Transmission (22–50 kV)"
    if "transmission" in t: return "Transmission (>60 kV)"
    return "All Voltages / Unspecified"

# -------------------------------
# EXTRACTION PIPELINE
# -------------------------------
print("🔍 Starting extraction...")

with pdfplumber.open(PDF_PATH) as pdf:
    total_pages = len(pdf.pages)
    print(f"📄 Total Pages: {total_pages}")

    for i, page in enumerate(pdf.pages, start=1):
        print(f"➡️ Processing Page {i}/{total_pages}")
        text = page.extract_text()
        plan, eff_date = None, None

        if text:
            plan_match = PLAN_PATTERN.search(text)
            if plan_match:
                plan = plan_match.group(1).strip()
            eff_match = EFFECTIVE_PATTERN.findall(text)
            if eff_match:
                eff_date = eff_match[-1].strip()

            # Extract text lines
            for line in text.split("\n"):
                if "$" in line or "per" in line.lower():
                    nums = MONEY_PATTERN.findall(line)
                    if nums:
                        while len(nums) < 4:
                            nums.append(None)
                        records.append({
                            "Page": i,
                            "Source": "Text",
                            "Service Plan": plan,
                            "Effective Date": eff_date,
                            "Charge Type": classify_charge_type(line),
                            "Voltage Level": classify_voltage(line),
                            "Rates": [n for n in nums if n],
                            "Raw Line": line.strip()
                        })
        else:
            # Fallback to OCR for scanned pages
            print(f"⚙️ OCR: Processing Page {i} (no text layer detected)")
            images = convert_from_path(PDF_PATH, first_page=i, last_page=i, fmt="png")
            for img in images:
                ocr_text = pytesseract.image_to_string(img)
                plan_match = PLAN_PATTERN.search(ocr_text)
                if plan_match:
                    plan = plan_match.group(1).strip()
                eff_match = EFFECTIVE_PATTERN.findall(ocr_text)
                if eff_match:
                    eff_date = eff_match[-1].strip()

                for line in ocr_text.split("\n"):
                    if "$" in line or "per" in line.lower():
                        nums = MONEY_PATTERN.findall(line)
                        if nums:
                            while len(nums) < 4:
                                nums.append(None)
                            records.append({
                                "Page": i,
                                "Source": "OCR",
                                "Service Plan": plan,
                                "Effective Date": eff_date,
                                "Charge Type": classify_charge_type(line),
                                "Voltage Level": classify_voltage(line),
                                "Rates": [n for n in nums if n],
                                "Raw Line": line.strip()
                            })

        # Try Camelot for structured tables
        try:
            tables = camelot.read_pdf(PDF_PATH, pages=str(i), flavor="stream")
            for t in tables:
                df_tab = t.df.replace(r"^\s*$", np.nan, regex=True).dropna(how="all")
                for _, row in df_tab.iterrows():
                    row_text = " | ".join([str(x) for x in row if pd.notna(x)])
                    if "$" in row_text:
                        nums = MONEY_PATTERN.findall(row_text)
                        if nums:
                            while len(nums) < 4:
                                nums.append(None)
                            records.append({
                                "Page": i,
                                "Source": "Table",
                                "Service Plan": plan,
                                "Effective Date": eff_date,
                                "Charge Type": classify_charge_type(row_text),
                                "Voltage Level": classify_voltage(row_text),
                                "Rates": [n for n in nums if n],
                                "Raw Line": row_text.strip()
                            })
        except Exception as e:
            print(f"⚠️ Skipping Camelot on Page {i}: {e}")

# -------------------------------
# CLEANUP
# -------------------------------
if not records:
    print("⚠️ No tariff data found.")
else:
    df = pd.DataFrame(records)
    df["Service Plan"] = df["Service Plan"].ffill()
    df["Effective Date"] = df["Effective Date"].ffill()

    # -------------------------------
    # DISPLAY CLEAN OUTPUT
    # -------------------------------
    print(f"\n✅ Extracted {len(df)} tariff lines from {df['Service Plan'].nunique()} service classifications.\n")

    for plan in df["Service Plan"].dropna().unique():
        subset = df[df["Service Plan"] == plan].reset_index(drop=True)
        effs = subset["Effective Date"].dropna().unique()
        print("="*110)
        print(f"📘 {plan}")
        print(f"📅 Effective Dates: {', '.join(effs) if len(effs)>0 else 'Not found'}")
        print(f"💰 Total Tariff Lines: {len(subset)}\n")
        display(HTML(subset[["Charge Type","Voltage Level","Rates","Raw Line"]].head(15).to_html(index=False)))


d:\utility-billing-ai\.venv\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


🔍 Starting extraction...
📄 Total Pages: 684
➡️ Processing Page 1/684


d:\utility-billing-ai\.venv\lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (176.35999999999999, 454.26800000000026, 447.5583999999999, 704.3766078378378)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


➡️ Processing Page 2/684
➡️ Processing Page 3/684
➡️ Processing Page 4/684
➡️ Processing Page 5/684
➡️ Processing Page 6/684
➡️ Processing Page 7/684
➡️ Processing Page 8/684
➡️ Processing Page 9/684
➡️ Processing Page 10/684
➡️ Processing Page 11/684
➡️ Processing Page 12/684
➡️ Processing Page 13/684
➡️ Processing Page 14/684
➡️ Processing Page 15/684
➡️ Processing Page 16/684
➡️ Processing Page 17/684
➡️ Processing Page 18/684
➡️ Processing Page 19/684
➡️ Processing Page 20/684
➡️ Processing Page 21/684
➡️ Processing Page 22/684
➡️ Processing Page 23/684
➡️ Processing Page 24/684
➡️ Processing Page 25/684
➡️ Processing Page 26/684
➡️ Processing Page 27/684
➡️ Processing Page 28/684
➡️ Processing Page 29/684


d:\utility-billing-ai\.venv\lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (97.986782, 508.0527268999999, 551.2541224744339, 661.0413269)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
d:\utility-billing-ai\.venv\lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (97.986782, 400.0527269, 550.6479184208102, 553.0413269000001)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


...

Charge Type,Voltage Level,Rates,Raw Line
Other,All Voltages / Unspecified,"[27, 2009]","INITIAL EFFECTIVE DATE: APRIL 27, 2009 SUPERSEDING REVISION:"


📘 SERVICE CLASSIFICATION NO. 11
📅 Effective Dates: March 15, 2013
💰 Total Tariff Lines: 1



Charge Type,Voltage Level,Rates,Raw Line
Other,All Voltages / Unspecified,"[1, 2013, 0]","INITIAL EFFECTIVE DATE: APRIL 1, 2013 SUPERSEDING REVISION: 0"


📘 SERVICE CLASSIFICATION NO. 11 (Continued)
📅 Effective Dates: March 15, 2013
💰 Total Tariff Lines: 4



Charge Type,Voltage Level,Rates,Raw Line
Other,All Voltages / Unspecified,"[1, 2013, 0]","INITIAL EFFECTIVE DATE: APRIL 1, 2013 SUPERSEDING REVISION: 0"
Other,All Voltages / Unspecified,"[1, 2013, 0]","INITIAL EFFECTIVE DATE: APRIL 1, 2013 SUPERSEDING REVISION: 0"
Other,All Voltages / Unspecified,"[1, 2013, 1]","INITIAL EFFECTIVE DATE: APRIL 1, 2013 SUPERSEDING REVISION: 1"
Other,All Voltages / Unspecified,"[1, 2013, 0]","INITIAL EFFECTIVE DATE: APRIL 1, 2013 SUPERSEDING REVISION: 0"


In [4]:
import re
import pdfplumber, pandas as pd, numpy as np
from IPython.display import display, HTML

PDF_PATH = "National Grid Tariff-New York.pdf"

# -------- Better regex patterns --------
PLAN_PATTERN = re.compile(r"(Service\s+Class(?:ification)?\s+No\.?\s*\d+[A-Z\-]*.*)", re.IGNORECASE)
EFFECTIVE_PATTERN = re.compile(r"(?:Effective|Issued)\s*[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})", re.IGNORECASE)
# Match only real rates, not dates
RATE_PATTERN = re.compile(r"\$?\s?(?:\d{1,3}(?:,\d{3})*|\d+)(?:\.\d{1,4})?\s*(?=(per|\/)?\s*(kwh|kw|month|therm|charge|delivery|distribution|peak|off|super|$))", re.IGNORECASE)

def classify_charge_type(line):
    t = line.lower()
    if "per kwh" in t: return "Energy Charge"
    if "per kw" in t: return "Demand Charge"
    if "per month" in t or "customer charge" in t: return "Fixed Charge"
    if "reactive" in t or "rkva" in t: return "Power Factor Charge"
    if "delivery" in t or "distribution" in t: return "Distribution Charge"
    return "Other"

def classify_voltage(line):
    t = line.lower()
    if "secondary" in t: return "Secondary (<2.2 kV)"
    if "primary" in t: return "Primary (2.2–15 kV)"
    if "sub" in t: return "Sub-Transmission (22–50 kV)"
    if "transmission" in t: return "Transmission (>60 kV)"
    return "All Voltages / Unspecified"

records = []

with pdfplumber.open(PDF_PATH) as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        text = page.extract_text() or ""
        plan_match = PLAN_PATTERN.search(text)
        plan = plan_match.group(1).strip() if plan_match else None

        eff_match = EFFECTIVE_PATTERN.findall(text)
        eff_date = eff_match[-1].strip() if eff_match else None

        for line in text.split("\n"):
            if "$" in line or "per" in line.lower():
                # Extract numeric rates (avoid short numbers like "1" in dates)
                numeric_rates = re.findall(r"\$?\s?(?:\d{1,3}(?:,\d{3})*|\d+)(?:\.\d{1,4})?", line)
                numeric_rates = [r for r in numeric_rates if not re.match(r"^\d{1,2}$", r)]
                if numeric_rates:
                    while len(numeric_rates) < 4:
                        numeric_rates.append(None)
                    records.append({
                        "Page": i,
                        "Service Plan": plan,
                        "Effective Date": eff_date,
                        "Charge Type": classify_charge_type(line),
                        "Voltage Level": classify_voltage(line),
                        "Rates": tuple(numeric_rates[:4]),  # ✅ convert list → tuple
                        "Raw Line": line.strip()
                    })

# 🧹 Convert to DataFrame and clean
df = pd.DataFrame(records)
df = df[~df["Raw Line"].str.contains("Effective Date|REVISION", case=False, na=False)]

# Deduplicate safely now that Rates is hashable
df = df.drop_duplicates(subset=["Service Plan", "Effective Date", "Charge Type", "Rates"]).reset_index(drop=True)

# ✅ Display summary and preview
print(f"✅ Extracted {len(df)} clean rate lines across {df['Service Plan'].nunique()} service classifications.\n")

for plan in df["Service Plan"].dropna().unique():
    subset = df[df["Service Plan"] == plan]
    print("="*120)
    print(f"📘 {plan}")
    effs = subset["Effective Date"].dropna().unique()
    print(f"📅 Effective Dates: {', '.join(effs) if len(effs)>0 else 'Not found'}")
    print(f"💰 Total Tariff Lines: {len(subset)}")
    display(HTML(subset[["Charge Type","Voltage Level","Rates","Raw Line"]].head(15).to_html(index=False)))


✅ Extracted 649 clean rate lines across 33 service classifications.

📘 Service Classification No. 1 or Service Classification No. 2
📅 Effective Dates: Not found
💰 Total Tariff Lines: 3


Charge Type,Voltage Level,Rates,Raw Line
Other,All Voltages / Unspecified,"($20.00, $20.00, None, None)",establishing service or discontinuing service shall be assessed a charge of $20.00. The $20.00
Other,All Voltages / Unspecified,"(26.13, $20.00, None, None)",26.13.2 The $20.00 charge shall be applicable and limited to one charge per visit for the same
Other,All Voltages / Unspecified,"($20.00, None, None, None)",visit is exempt from the $20.00 charge.


📘 Service Classification No.12, Special Contract Rates, in either
📅 Effective Dates: March 15, 2018
💰 Total Tariff Lines: 1


Charge Type,Voltage Level,Rates,Raw Line
Other,All Voltages / Unspecified,"(34.6, 3.4, 1.6, None)","34.6.3.4.1.6 For the billing period, aggregate the apportioned NYPA and Company"


📘 Service Classification No. 7, except an
📅 Effective Dates: July 1, 2021
💰 Total Tariff Lines: 2


...

Charge Type,Voltage Level,Rates,Raw Line
Other,All Voltages / Unspecified,"( 3, None, None, None)",3. retain the records for a 3-year period.
Fixed Charge,All Voltages / Unspecified,"( 10, None, None, None)","systems, not to exceed 10 hours per month or as otherwise agreed to by the Company in the"
Other,All Voltages / Unspecified,"( 5, None, None, None)","representatives. Customers operating an on-site generator unit less than 5 MW may apply for service using the SIR,"
Other,All Voltages / Unspecified,"( 25, None, None, None)",Rates are established on a calendar month basis and will only be prorated if the billing period is less than 25 days or
Fixed Charge,All Voltages / Unspecified,"($17.33, None, None, None)",Customer Charge: $17.33
Distribution Charge,All Voltages / Unspecified,"($28.81, None, None, None)",Distribution Delivery $28.81
Distribution Charge,All Voltages / Unspecified,"($0.2448, None, None, None)",Distribution Delivery $0.2448
Fixed Charge,All Voltages / Unspecified,"($19.00, None, None, None)",Customer Charge: $19.00
Distribution Charge,All Voltages / Unspecified,"($5.49, None, None, None)",Distribution Delivery $5.49
Distribution Charge,All Voltages / Unspecified,"($0.4610, None, None, None)",Distribution Delivery $0.4610


📘 SERVICE CLASSIFICATION NO. 7 (CONTINUED)
📅 Effective Dates: Not found
💰 Total Tariff Lines: 2


Charge Type,Voltage Level,Rates,Raw Line
Distribution Charge,All Voltages / Unspecified,"($0.4017, $0.4017, $0.0000, $0.0000)",Distribution Delivery $0.4017 $0.4017 $0.0000 $0.0000
Other,All Voltages / Unspecified,"( 34.7, 3.3, None, None)","determined in Rule 34.7.3.3.2 for EJP, and the number of on-peak or super-peak days"


📘 SERVICE CLASSIFICATION NO. 12
📅 Effective Dates: December 16, 2011
💰 Total Tariff Lines: 1


Charge Type,Voltage Level,Rates,Raw Line
Other,All Voltages / Unspecified,"( 4.6, 12, None, None)","Section 4.6 of this Service Classification No. 12 are not representative of the customer’s operations, the"


In [5]:
import pdfplumber, re, pandas as pd

PDF_PATH = "National Grid Tariff-New York.pdf"

# ---------- Helper Functions ----------
def extract_rate_lines(block):
    """Extract lines containing numeric rates or tariffs."""
    rate_lines = []
    for line in block.split("\n"):
        if re.search(r"\$?\d+\.\d+", line):
            rate_lines.append(line.strip())
    return rate_lines

def extract_field(pattern, text):
    m = re.search(pattern, text, re.IGNORECASE)
    return m.group(1).strip() if m else None

# ---------- Extraction by Service Classification ----------
records = []

with pdfplumber.open(PDF_PATH) as pdf:
    text = "\n".join(page.extract_text() or "" for page in pdf.pages)

# Split by service classification sections
sections = re.split(r"SERVICE\s+CLASSIFICATION\s+NO\.\s*", text, flags=re.IGNORECASE)
for sec in sections[1:]:
    sc_number = sec.split("\n", 1)[0].split()[0]  # e.g., '1', '2D', '3A'
    block = sec.strip()

    # Common metadata
    effective_date = extract_field(r"Effective\s*(?:Date|on)?[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})", block)
    applicable = extract_field(r"APPLICABLE\s+TO\s+USE\s+OR\s+ELIGIBILITY[:\-]?\s*(.*)", block)
    monthly_min = extract_field(r"MONTHLY\s+MINIMUM\s+CHARGE[:\-]?\s*\$?([\d,\.]+)", block)
    reactive = extract_field(r"REACTIVE\s+DEMAND\s+CHARGE.*?(\$?\d+\.\d+)", block)

    # ---------- Example Patterns per SC ----------
    if "2D" in sc_number.upper():
        # Demand + On/Off/Super Peak
        demand_lines = re.findall(r"Distribution.*?\$[\d\.\s]+", block)
        energy_lines = re.findall(r"(On|Off|Super)\s*Peak.*?\$[\d\.\s]+", block)
        delivery_voltage = re.findall(r"(Secondary|Primary|Subtransmission|Transmission).*?\$[\d\.\s]+", block)
        records.append({
            "Service Classification": f"SC-{sc_number}",
            "Effective Date": effective_date,
            "Applicable To": applicable,
            "Monthly Minimum": monthly_min,
            "Reactive Charge": reactive,
            "Rate Period": "Demand / Energy (Peak Types)",
            "Rate Details": demand_lines + energy_lines,
            "Delivery Voltages": delivery_voltage
        })

    elif "1" == sc_number:
        # Residential simple tariff
        base_rate = extract_field(r"Per\s*kWh[:\-]?\s*\$?([\d\.]+)", block)
        cust_charge = extract_field(r"(?:Customer|Basic)\s+Charge.*?\$?([\d\.]+)", block)
        records.append({
            "Service Classification": "SC-1 (Residential)",
            "Effective Date": effective_date,
            "Applicable To": applicable,
            "Monthly Minimum": monthly_min,
            "Rate Period": "All Hours",
            "Standard Tariff Type": "Energy Charge",
            "Rate": base_rate,
            "Customer Charge": cust_charge
        })

    elif "3A" in sc_number.upper():
        # Large General Service – Multi Voltage
        delivery_voltage = re.findall(r"(Secondary|Primary|Subtransmission|Transmission).*?\$[\d\.\s]+", block)
        records.append({
            "Service Classification": f"SC-{sc_number}",
            "Effective Date": effective_date,
            "Delivery Voltage Rates": delivery_voltage,
            "Rate Period": "On/Off/Super Peak",
            "Applicable To": applicable
        })

# ---------- Create DataFrame ----------
df = pd.DataFrame(records)
display(df.head(10))


,Service Classification,Effective Date,Applicable To,Monthly Minimum,Rate Period,Standard Tariff Type,Rate,Customer Charge,Delivery Voltage Rates,Reactive Charge,Rate Details,Delivery Voltages
0,SC-1 (Residential),None,None,None,All Hours,Energy Charge,None,None,NaN,NaN,NaN,NaN
1,SC-1 (Residential),None,None,None,All Hours,Energy Charge,None,None,NaN,NaN,NaN,NaN
2,"SC-3A,",None,None,NaN,On/Off/Super Peak,NaN,NaN,NaN,[],NaN,NaN,NaN
3,SC-1 (Residential),None,None,None,All Hours,Energy Charge,None,None,NaN,NaN,NaN,NaN
4,"SC-3A,",None,None,NaN,On/Off/Super Peak,NaN,NaN,NaN,[],NaN,NaN,NaN
5,SC-3A,None,None,NaN,On/Off/Super Peak,NaN,NaN,NaN,[],NaN,NaN,NaN
6,SC-3A,None,None,NaN,On/Off/Super Peak,NaN,NaN,NaN,[],NaN,NaN,NaN
7,SC-3A,None,None,NaN,On/Off/Super Peak,NaN,NaN,NaN,[],NaN,NaN,NaN
8,SC-3A,"SEPTEMBER 1, 2025",None,NaN,On/Off/Super Peak,NaN,NaN,NaN,[],NaN,NaN,NaN
9,"SC-2D,","SEPTEMBER 1, 2025",None,None,Demand / Energy (Peak Types),NaN,NaN,NaN,NaN,NaN,[],[]
